Орги попросили прогнать код на их тестовых даных и предсказать их. 

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter('ignore')

In [5]:
#train, test = train_test_split(data,random_state = 1,test_size = 0.2)
train = pd.read_csv('X_train.csv')
test = pd.read_csv('X_final_test.csv')
uuid = test['uuid']
del test['uuid']
#y_test = test['reting']
y = train['reting']
index = train.shape[0]
test['reting'] = -1
#разделили на трейн и тест, теперь опять ссоединим вместе, чтобы сразу чистить и генерить фичи на всем датасете
df = train.append(test,ignore_index=True)

In [8]:
#можно как-то использовать колонку property, но так как это отборочный этап, и что-то сложное делать не будем, просто удалим колонку
del df['property']
# userName зашифрован в MD5, можно конечно попробовать расшифровать процентов 40% ников, по никам нагенерить признаки, вроди пола
#и тд но мы это тоже делать не будем

In [9]:
#нагенерим фичи, связанные с датой
df['date'] = pd.to_datetime(df['date'])
df['day'] = df['date'].apply(lambda x: x.day)
df['weekday'] = df['date'].apply(lambda x: x.weekday())
df['month'] = df['date'].apply(lambda x: x.month)
df['year'] = df['date'].apply(lambda x: x.year)
del df['date']

In [10]:
df['commentNegative'] = df['commentNegative'].fillna('')
df['commentPositive'] = df['commentPositive'].fillna('')
df['commentPositive_len'] = df['commentPositive'].str.len()
df['commentNegative_len'] = df['commentNegative'].str.len()
df['comment_len'] = df['comment'].str.len()

In [11]:
#так как commentNegative и commentNegative во многих отзывах пропущены, то просто склеим их к коменту и удалим
#в роли флага, был ли негативный или позитинвый коммент, будет выступать commentNegative_len и commentPositive_len
df['comment'] = df['comment'] + ' œ ' + df['commentPositive'] + ' å ' +  df['commentNegative']

In [12]:
df['comment_len_all'] = df['comment'].str.len()

In [13]:
del df['commentNegative']
del df['commentPositive']

In [14]:
def average(col,alpha = 10,y_target = 'reting'):
    ans = pd.Series(index = df.index)#создали пустой массив, с индексами,и размерностью df
    X_train = df[df['reting'] != -1][[col,y_target]]#создали новый датафрейм из друх колонок:колонки col and y_target, берем только трейн данные
    X_test = df[df['reting'] == -1][col]#копируем колонку col тест данных
    mean_by_col = X_train.groupby(col).mean()[y_target]#для каждого уникального значения колонки col найдем средний таргет
    counts = X_train[col].value_counts()#подсчитаем количество каждого уникального значения col
    mean = X_train[y_target].mean()# запомним глобальное среднее таргета
    ans_test = X_test.apply(lambda x: (mean_by_col[x]*counts[x] + alpha*mean) / (counts[x] + alpha)  if x in counts.index else mean)
    #строчка выше вычисляет формулу из видео
    ans[ans_test.index] = ans_test#на нужные инлексы запишем наш полученный результат.
    #это мы зная весь трейн, вычислили сркднее по таргету для теста. Теперь надо получить также среднее для трейна, но чтобы не переобучаться
    #мы должны использовать кросс валидацию.Перемешаем и Разобьем трецн на 5 частей. на 4 частях будем обучатьсч, а на пятой предсказывать
    kf = KFold(n_splits=5,random_state=42,shuffle = True)
    for train_i,test_i in kf.split(X_train):
        #train_i - индексы, которые попали в первые 4 части и на них будем обучаться
        #test_i -индексы для которых будем делать предсказания
        #теперь делаем все тоже самое, что и выше, когда мы предсказывали среднее для теста по трейну
        X = X_train.iloc[train_i]
        X_t = X_train.iloc[test_i,0]
        mean_by_col = X.groupby(col).mean()[y_target]
        counts = X[col].value_counts()
        mean = X[y_target].mean()
        ans_test = X_t.apply(lambda x: (mean_by_col[x]*counts[x] + alpha*mean) / (counts[x] + alpha)  if x in counts.index else mean)
        ans[ans_test.index] = ans_test
    return ans

In [15]:
#как учил Семенов, делаем mean target для всех колонок. Двойную кросс валидацию делать не будем, так как и так сойдет =) 

In [16]:
col_to_mean_target = ['sku','categoryLevel1Id','categoryLevel2Id','brandId','userName','day','weekday','month',
                     'year','commentPositive_len','commentNegative_len','comment_len','comment_len_all']

In [17]:
for i in col_to_mean_target:
    df[i + '_mean_target'] = average(i)

Теперь еще одна сложная штука для понимания, но простая в реализации. Это субъективное восприятие товара человеком. Бывают люди гурманы, которые считают что все товары говно, и всегда ставят оценку к товару ниже, чем средняя оценка. А есть наоборот, люди, которым все товары нравятся и они ставят завышенную оценку. Создадим новую переменную: разницу между оценкой, которую поставил человек и средней оценкой товара

In [18]:
df['tmp'] = df['reting'] - df['categoryLevel2Id_mean_target']

И теперь применим среднее по таргету для кажого человека, при этом таргетом у нас как раз будет tmp переменная. Что мы получили? Получили на сколько в среднем отзыв человека ниже/выше среднего отзыва о товаре. Ну и добавим новую фичу: предпологаемая оценка товара человеком.

In [19]:
df['delta_score'] = average('userName',alpha=1,y_target='tmp')
df['predict'] = df['categoryLevel2Id_mean_target'] + df['delta_score']

Теперь займемся самими текстами. Не будем использовать модные word2vec, нейронки, LSTM, а возьмем добрый, проверенный временем мешок слов tfidf

Так как организарты не обозначили конкретную метрику, то будем использовать MSE, так как у нас отзывы упорядочены. В конце будем просто округлять до целого.

с помощью GridSearchCV найдем оптимальные параметры для TfidfVectorizer

In [20]:
%%time
for i in range(1,4):
    tf = TfidfVectorizer(ngram_range=(1,i))
    summary = tf.fit_transform(df['comment'])
    grid ={'alpha':[0.1]}
    gr = GridSearchCV(Ridge(),grid,cv = 2,scoring='neg_mean_squared_error',n_jobs=-1)
    gr.fit(summary[:index],y)
    print i,gr.grid_scores_

1 [mean: -1.07009, std: 0.00022, params: {'alpha': 0.1}]
2 [mean: -0.90761, std: 0.01000, params: {'alpha': 0.1}]
3 [mean: -0.97064, std: 0.01215, params: {'alpha': 0.1}]
CPU times: user 31.1 s, sys: 1.45 s, total: 32.5 s
Wall time: 38.1 s


In [21]:
%%time
for i in range(100000,1100000,100000):
    tf = TfidfVectorizer(ngram_range=(1,2),max_features =  int(i))
    summary = tf.fit_transform(df['comment'])
    grid ={'alpha':[0.1]}
    gr = GridSearchCV(Ridge(),grid,cv = 2,scoring='neg_mean_squared_error',n_jobs=-1)
    gr.fit(summary[:index],y)
    print i,gr.grid_scores_

100000 [mean: -0.89040, std: 0.00150, params: {'alpha': 0.1}]
200000 [mean: -0.88271, std: 0.00148, params: {'alpha': 0.1}]
300000 [mean: -0.88949, std: 0.00558, params: {'alpha': 0.1}]
400000 [mean: -0.89930, std: 0.00838, params: {'alpha': 0.1}]
500000 [mean: -0.90762, std: 0.01000, params: {'alpha': 0.1}]
600000 [mean: -0.90762, std: 0.01001, params: {'alpha': 0.1}]
700000 [mean: -0.90762, std: 0.01000, params: {'alpha': 0.1}]
800000 [mean: -0.90761, std: 0.00999, params: {'alpha': 0.1}]
900000 [mean: -0.90761, std: 0.00999, params: {'alpha': 0.1}]
1000000 [mean: -0.90762, std: 0.01001, params: {'alpha': 0.1}]
CPU times: user 1min 37s, sys: 5.17 s, total: 1min 42s
Wall time: 2min 5s


In [23]:
%%time
for i in range(100000,300000,10000):
    tf = TfidfVectorizer(ngram_range=(1,2),max_features =  int(i))
    summary = tf.fit_transform(df['comment'])
    grid ={'alpha':[0.1]}
    gr = GridSearchCV(Ridge(),grid,cv = 2,scoring='neg_mean_squared_error',n_jobs=-1)
    gr.fit(summary[:index],y)
    print i,gr.grid_scores_

100000 [mean: -0.89041, std: 0.00152, params: {'alpha': 0.1}]
110000 [mean: -0.88852, std: 0.00226, params: {'alpha': 0.1}]
120000 [mean: -0.88726, std: 0.00251, params: {'alpha': 0.1}]
130000 [mean: -0.88561, std: 0.00177, params: {'alpha': 0.1}]
140000 [mean: -0.88450, std: 0.00103, params: {'alpha': 0.1}]
150000 [mean: -0.88318, std: 0.00034, params: {'alpha': 0.1}]
160000 [mean: -0.88233, std: 0.00000, params: {'alpha': 0.1}]
170000 [mean: -0.88252, std: 0.00041, params: {'alpha': 0.1}]
180000 [mean: -0.88264, std: 0.00068, params: {'alpha': 0.1}]
190000 [mean: -0.88224, std: 0.00134, params: {'alpha': 0.1}]
200000 [mean: -0.88275, std: 0.00148, params: {'alpha': 0.1}]
210000 [mean: -0.88292, std: 0.00192, params: {'alpha': 0.1}]
220000 [mean: -0.88306, std: 0.00234, params: {'alpha': 0.1}]
230000 [mean: -0.88369, std: 0.00270, params: {'alpha': 0.1}]
240000 [mean: -0.88461, std: 0.00350, params: {'alpha': 0.1}]
250000 [mean: -0.88524, std: 0.00366, params: {'alpha': 0.1}]
260000 [

Нашли оптимальные параметры для  tfidf: ngram_range=(1,2),max_features = 160000 

Мы создали новые фичи. Возьмем фичи, которые увеличивают score. И отнормируем их, так как мы будем использовать линейную регрессию с l2 регуляризацией, ее также называют Ridge.

In [27]:
data = df[['delta_score','predict','comment_len_all_mean_target','comment_len_mean_target',
    'commentNegative_len_mean_target','commentPositive_len_mean_target']]

In [28]:
ss = StandardScaler()
data_s=ss.fit_transform(data)

In [29]:
tf = TfidfVectorizer(ngram_range=(1,2),max_features = 160000)
summary = tf.fit_transform(df['comment'])

In [30]:
X_test = hstack([summary[index:,],data_s[index:,]])
X_train = hstack([summary[:index,],data_s[:index,]])

Для проверки результатов, отбора фич и настройки параметра alpha, я использовал кросс-валидацию. Для этого использовал GridSearchCV. Так как данные хорошо были перемешаны, то достаточно было использовать всего два фолда, т е разбивать трейн на две части, на одной обучать, а на другой проверять.

In [31]:
%%time
grid ={'alpha':np.linspace(0.000000000001,0.1,15)}
gr = GridSearchCV(Ridge(),grid,cv = 2,scoring='neg_mean_squared_error',n_jobs=-1)
gr.fit(X_train,y)

CPU times: user 12.3 s, sys: 352 ms, total: 12.6 s
Wall time: 2min 11s


In [32]:
gr.grid_scores_

[mean: -0.87825, std: 0.00780, params: {'alpha': 9.9999999999999998e-13},
 mean: -0.87872, std: 0.00782, params: {'alpha': 0.007142857143785715},
 mean: -0.87896, std: 0.00764, params: {'alpha': 0.014285714286571429},
 mean: -0.87943, std: 0.00766, params: {'alpha': 0.021428571429357141},
 mean: -0.87988, std: 0.00791, params: {'alpha': 0.028571428572142857},
 mean: -0.88025, std: 0.00781, params: {'alpha': 0.035714285714928573},
 mean: -0.88062, std: 0.00769, params: {'alpha': 0.042857142857714282},
 mean: -0.88110, std: 0.00771, params: {'alpha': 0.050000000000499999},
 mean: -0.88160, std: 0.00779, params: {'alpha': 0.057142857143285715},
 mean: -0.88197, std: 0.00770, params: {'alpha': 0.064285714286071438},
 mean: -0.88224, std: 0.00789, params: {'alpha': 0.071428571428857154},
 mean: -0.88262, std: 0.00800, params: {'alpha': 0.078571428571642871},
 mean: -0.88310, std: 0.00782, params: {'alpha': 0.085714285714428573},
 mean: -0.88348, std: 0.00772, params: {'alpha': 0.09285714285

Тут можно много что еще делать, юзать xgb, нейронку, потом запилить стекинг, но так как это тестовое задание, и в github вбив в поиск mvideo, можно посмотреть решения других участников, то понятно, что участников не так много, так что можно остановиться на этом. Теперь просто предскажем нашей моделью тестовые данные.

In [33]:
rg = Ridge(alpha=9.9999999999999998e-13)
rg.fit(X_train,y)

Ridge(alpha=1e-12, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [34]:
ans = rg.predict(X_test)

In [37]:
test['reting'] = ans
test['uuid'] = uuid

In [39]:
test.to_csv('ans.csv')

In [41]:
test.head()

,sku,categoryLevel1Id,categoryLevel2Id,brandId,property,userName,date,comment,commentNegative,commentPositive,reting,uuid
0,20003635,406,4060501,62,"[{719: 'a9b7ba70783b617e9998dc4dd82eb3c5'}, {1...",e033ded0ac328c61d6b007139ec4e219,2012-04-16,"Согласна с Сашей и Наташей. Самая лучшая,самая...",NaN,NaN,4.901586,e0880553-b7f1-4eaa-a800-966133bda683
1,20000782,411,4110104,13,"[{34: '9ce895413ebdf6b6dcb69b07dc782591'}, {36...",9f269ce1c8e74e7b1e63f55b856068cd,2010-12-11,"Хороший пылесос. Вроде качественный. Да, шумно...",NaN,NaN,4.753868,30039f3f-e719-4b47-b588-9a05fd11a799
2,20021828,413,4130401,63,"[{34: 'f982777489055c6563d68c005fd24aad'}, {36...",dc9731cd5ea3ab7e16401cc678d046a5,2012-11-02,"Пользовалась им почти 2 года, без нареканий. Н...",NaN,NaN,3.651724,352a5abc-10f4-4668-ae5a-a4285413811a
3,20027435,413,4130301,678,"[{539: '47c7a779688d5f67c55a42811da13210'}, {3...",c9b5ecdf0b853aab02103542187ea208,2016-04-22,До этого у меня был триммер всего с одной наса...,NaN,NaN,4.650282,bfe4564d-944c-4680-b1ce-14e5486ada52
4,20004128,413,4130401,1466,"[{162: 'f932bed2d12442d21507b51d22b88dd7'}, {2...",32bf3da6fec001612d3e5048d87c5c64,2009-10-22,Удобны в использовании. Волосы становятся глад...,NaN,NaN,4.905330,13547ee0-3400-422f-b831-528caeda9c69
